In [1]:
import torch, numpy as np
from multitask.framework1.multiscale_fusion import StandardMultiScaleFusion, SmartMultiScaleFusion
from multitask.framework1.model import MultiTaskFaceAnalysisModel
from backbones.backbones import get_backbone
from multitask.framework1.subnets import FaceRecognitionEmbeddingSubnet, AgeEstimationSubnet, GenderRecognitionSubnet, EmotionRecognitionSubnet, RaceRecognitionSubnet, AttributeRecognitionSubnet, PoseEstimationSubnet

/home/thoal/anaconda3/envs/pytorch/lib/python3.13/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
backbone = get_backbone('swin_v2_t')

/home/thoal/anaconda3/envs/pytorch/lib/python3.13/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
multiscale_features = backbone(torch.randn(size = (50, 3, 112, 112)))

In [4]:
fusion_type = 'smart'

In [5]:
subnets = [
    FaceRecognitionEmbeddingSubnet(),
    EmotionRecognitionSubnet(multiscale_fusion_type=fusion_type),
    AgeEstimationSubnet(multiscale_fusion_type=fusion_type),
    GenderRecognitionSubnet(multiscale_fusion_type=fusion_type),
    RaceRecognitionSubnet(multiscale_fusion_type=fusion_type),
    AttributeRecognitionSubnet(multiscale_fusion_type=fusion_type),
    PoseEstimationSubnet(multiscale_fusion_type=fusion_type)
]

embedding, embedding_norm = subnets[0](multiscale_features)
print(embedding.shape, embedding_norm.shape)

torch.Size([50, 512]) torch.Size([50, 1])


In [6]:
for subnet in subnets[1:]:
    output = subnet(multiscale_features)
    count = 0
    for parameter in subnet.parameters():
        count += parameter.numel()
    
    print(count)

1332049
1380784
1328971
1331023
1348978
1331536
